In [18]:
import os
import json
import pandas as pd
import geopandas as gpd
from shapely.geometry.polygon import Polygon
from shapely.geometry.multipolygon import MultiPolygon

import json
import matplotlib as mpl
import pylab as plt
import string

from bokeh.io import output_file, show, output_notebook, export_png
from bokeh.models import ColumnDataSource, GeoJSONDataSource, LogColorMapper, ColorBar, HoverTool
from bokeh.plotting import figure, save, output_file
from bokeh.palettes import brewer
from bokeh.tile_providers import CARTODBPOSITRON, get_provider

## Read in the data

In [171]:
# Define the filepaths
home_dir = os.path.expanduser("~")
data_fp = "iec/shrug_covid"

# Read in the COVID data
df = pd.read_stata(os.path.join(home_dir, data_fp, "hospitals_dist_export.dta"))

# Read in the pc11 district shapefile
gdf = gpd.read_file(os.path.join(home_dir, "iec1", "gis", "pc11", "pc11-district.shp"))

# Rename columns to match pc11 keys
gdf = gdf.rename(columns = {"pc11_s_id": "pc11_state_id",
                            "pc11_d_id": "pc11_district_id"})


data = gdf.merge(df, left_on=["pc11_state_id", "pc11_district_id"], right_on=["pc11_state_id", "pc11_district_id"])

# convert multipoloygon states into individual polygons
data = data.explode()

# reduce resolution of polygons to make loading faster
data['geometry'] = data['geometry'].simplify(tolerance=0.01)

# set crs
# data.to_crs = 'epsg:3857'

data['pc11_state_name'] = [string.capwords(x) for x in data['pc11_state_name']]
data['pc11_district_name'] = [string.capwords(x) for x in data['pc11_district_name']]

## Plot the Data

In [179]:
def get_geodatasource(gdf):    
    """Get getjsondatasource from geopandas object"""
    json_data = json.dumps(json.loads(gdf.to_json()))
    return GeoJSONDataSource(geojson = json_data)

def bokeh_plot_map(gdf, column=None, title=''):
    """Plot bokeh map from GeoJSONDataSource """

    geosource = get_geodatasource(gdf)
    palette = brewer['OrRd'][8]
    palette = palette[::-1]
    vals = gdf[column]
    
    #Instantiate LogColorMapper that maps numbers in a range, into a sequence of colors.
    color_mapper = LogColorMapper(palette = palette, low = vals.min(), high = vals.max())
    color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8, width=500, height=20,
                         location=(0,0), orientation='horizontal')

    # define tools
    tools = 'wheel_zoom, pan, reset'
    
    # define basemap
    tile_provider = get_provider(CARTODBPOSITRON)

    # create figure
    p = figure(title = title, plot_height=850 , plot_width=850,
               x_axis_type="mercator", y_axis_type="mercator",
               toolbar_location='right', tools=tools)
    #p.add_tile(tile_provider)
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None
    

    #Add patch renderer to figure
    p.patches('xs','ys', source=geosource, fill_alpha=1, line_width=0.5, line_color='black',  
              fill_color={'field' :column , 'transform': color_mapper})
    
    #create hover tool
    hover_tool = HoverTool(
        tooltips=[
            ("State", "@pc11_state_name"),
            ("District", "@pc11_district_name"),
            ("Number of Hospital Beds", "@dlhs4_perk_beds_pubpriv{1.11}")
        ])
    p.tools.append(hover_tool)
        
    #Specify figure layout.
    p.add_layout(color_bar, 'below')
    
    return p

In [180]:
p = bokeh_plot_map(data, column="dlhs4_perk_beds_pubpriv")

In [181]:
output_file(os.path.join(home_dir, data_fp, "bokeh-test.html"), title="Number of Hospital Beds per 1000")
save(p)

'/dartfs-hpc/rc/home/y/f00473y/iec/shrug_covid/bokeh-test.html'